In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime  # Corrected import for timestamp functionality

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Image transformations (matching your training setup)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset for loading images with labels
class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path, label

# Modified ResNet18 model to support CAM
class ResNet18ForCAM(nn.Module):
    def __init__(self, num_classes=2, dropout_rate=0.2):
        super(ResNet18ForCAM, self).__init__()
        self.backbone = models.resnet18(weights=None)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.backbone(x)
        pooled = self.global_pool(features)
        return self.classifier(pooled)

    def get_features(self, x):
        return self.backbone(x)

# Load saved model
def load_model(model_path, device):
    try:
        model = ResNet18ForCAM(num_classes=2, dropout_rate=0.2)
        checkpoint = torch.load(model_path, map_location=device)
        if 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
        else:
            model.load_state_dict(checkpoint)
        model.to(device)
        model.eval()
        return model
    except Exception as e:
        print(f"Error loading model {model_path}: {e}")
        return None

# Generate CAM heatmap using Grad-CAM (with added threshold)
def generate_cam(model, image_tensor, target_class, device):
    model.eval()
    
    features = None
    gradients = None
    
    def hook_features(module, input, output):
        nonlocal features
        features = output
    
    def hook_gradients(module, grad_in, grad_out):
        nonlocal gradients
        gradients = grad_out[0]
    
    # Register hooks on the last conv layer
    try:
        handle_features = model.backbone[-1][-1].conv2.register_forward_hook(hook_features)
        handle_gradients = model.backbone[-1][-1].conv2.register_backward_hook(hook_gradients)
    except:
        # Alternative hook registration if structure differs
        handle_features = model.backbone[-1].conv2.register_forward_hook(hook_features)
        handle_gradients = model.backbone[-1].conv2.register_backward_hook(hook_gradients)
    
    # Forward pass
    output = model(image_tensor)
    model.zero_grad()
    
    # Backward pass for target class
    one_hot = torch.zeros_like(output)
    one_hot[0][target_class] = 1
    output.backward(gradient=one_hot, retain_graph=True)
    
    # Compute Grad-CAM
    weights = torch.mean(gradients, dim=[2, 3], keepdim=True)
    cam = torch.sum(weights * features, dim=1).squeeze()
    cam = F.relu(cam)
    cam = F.interpolate(cam.unsqueeze(0).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False)
    cam = cam.squeeze().detach().cpu().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
    cam[cam < 0.2] = 0  # Threshold to suppress weak activations
    
    # Clean up hooks
    handle_features.remove()
    handle_gradients.remove()
    
    return cam

# Visualize and save CAM (with corrected timestamp handling)
def visualize_and_save_cam(original_path, cam, prediction, actual_label, output_dir, filename, show_plots=False):
    original = cv2.imread(original_path)
    if original is None:
        print(f"Could not load image: {original_path}")
        return
        
    original = cv2.resize(original, (224, 224))
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    overlaid = cv2.addWeighted(original, 0.6, heatmap, 0.4, 0)
    
    # Ensure absolute paths and print for debugging
    abs_output_dir = os.path.abspath(output_dir)
    os.makedirs(abs_output_dir, exist_ok=True)
    print(f"Saving to directory: {abs_output_dir}")
    
    # Save components with unique timestamps to avoid conflicts (with try-except for safety)
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    except Exception as e:
        print(f"Timestamp error: {e}. Using fallback.")
        timestamp = "no_timestamp"
        
    original_path_save = os.path.join(abs_output_dir, f"{filename}_{timestamp}_original.png")
    heatmap_path_save = os.path.join(abs_output_dir, f"{filename}_{timestamp}_heatmap.png")
    overlaid_path_save = os.path.join(abs_output_dir, f"{filename}_{timestamp}_cam_overlay.png")
    
    cv2.imwrite(original_path_save, original)
    print(f"Saved original to: {original_path_save}")
    
    cv2.imwrite(heatmap_path_save, heatmap)
    print(f"Saved heatmap to: {heatmap_path_save}")
    
    cv2.imwrite(overlaid_path_save, overlaid)
    print(f"Saved overlay to: {overlaid_path_save}")
    
    # Optional: Display
    if show_plots:
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.title('Original Image')
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.title('Heatmap')
        plt.imshow(cam, cmap='jet')
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.title(f'Predicted: {prediction}\nActual: {actual_label}')
        plt.imshow(cv2.cvtColor(overlaid, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# Enhanced function to generate CAMs for all body parts with normal/abnormal separation
def generate_cams_for_all_body_parts(image_csv, root_dir, model_dir, output_dir, 
                                    num_normal=5, num_abnormal=5, show_plots=False):
    """
    Generate CAMs for all body parts, including both normal and abnormal cases
    
    Args:
        image_csv: Path to CSV file with image paths and labels
        root_dir: Root directory containing MURA images
        model_dir: Directory containing saved model files
        output_dir: Directory to save CAM visualizations
        num_normal: Number of normal cases to process per body part
        num_abnormal: Number of abnormal cases to process per body part
        show_plots: Whether to display plots during processing
    """
    
    # All body parts in MURA dataset
    body_parts = ['XR_ELBOW', 'XR_FINGER', 'XR_FOREARM', 'XR_HAND', 
                  'XR_HUMERUS', 'XR_SHOULDER', 'XR_WRIST']
    
    if not os.path.exists(image_csv):
        print(f"CSV file not found: {image_csv}")
        return
    
    # Load the CSV data
    data = pd.read_csv(image_csv, header=None, names=['path', 'label'])
    
    for body_part in body_parts:
        print(f"\n{'='*60}")
        print(f"Processing {body_part}")
        print(f"{'='*60}")
        
        # Check if model exists for this body part
        model_path = os.path.join(model_dir, f'resnet18_{body_part}_final.pth')
        if not os.path.exists(model_path):
            print(f"❌ Model not found for {body_part}: {model_path}")
            continue
        
        # Load model
        model = load_model(model_path, device)
        if model is None:
            continue
            
        print(f"✅ Model loaded successfully for {body_part}")
        
        # Filter data for this body part
        body_part_data = data[data['path'].str.contains(body_part)]
        
        if len(body_part_data) == 0:
            print(f"⚠️ No data found for {body_part}")
            continue
        
        # Separate normal and abnormal cases
        normal_data = body_part_data[body_part_data['label'] == 0].head(num_normal * 3)  # Get more to account for missing files
        abnormal_data = body_part_data[body_part_data['label'] == 1].head(num_abnormal * 3)
        
        # Process both normal and abnormal cases
        for case_type, case_data, num_cases in [('normal', normal_data, num_normal), 
                                               ('abnormal', abnormal_data, num_abnormal)]:
            
            print(f"\n📊 Processing {case_type} cases for {body_part}...")
            
            image_paths = []
            labels = []
            
            # Collect image paths and labels
            processed_count = 0
            for _, row in case_data.iterrows():
                if processed_count >= num_cases:
                    break
                    
                study_path = os.path.join(root_dir, row['path'].replace('MURA-v1.1/', ''))
                if os.path.exists(study_path):
                    try:
                        for img_name in os.listdir(study_path):
                            if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                                image_paths.append(os.path.join(study_path, img_name))
                                labels.append(row['label'])
                                processed_count += 1
                                if processed_count >= num_cases:
                                    break
                    except PermissionError:
                        continue
            
            if not image_paths:
                print(f"⚠️ No {case_type} images found for {body_part}")
                continue
            
            print(f"📁 Found {len(image_paths)} {case_type} images for {body_part}")
            
            # Create dataset and dataloader
            dataset = ImageDataset(image_paths, labels, transform=transform)
            loader = DataLoader(dataset, batch_size=1, shuffle=False)
            
            # Generate CAMs
            for i, (image, img_path, actual_label) in enumerate(tqdm(loader, 
                                                                    desc=f"Generating {case_type} CAMs")):
                try:
                    image = image.to(device)
                    output = model(image)
                    _, predicted = torch.max(output, 1)
                    pred_class = predicted.item()
                    pred_label = 'Abnormal' if pred_class == 1 else 'Normal'
                    actual_label_str = 'Abnormal' if actual_label.item() == 1 else 'Normal'
                    
                    # Generate CAM for both predicted class and actual class
                    for target_class, class_name in [(pred_class, 'predicted'), (actual_label.item(), 'actual')]:
                        cam = generate_cam(model, image, target_class, device)
                        
                        # Create output directory for this body part and case type
                        case_output_dir = os.path.join(output_dir, body_part, case_type, class_name)
                        filename = f"{body_part}_{case_type}_{class_name}_{i}"
                        
                        visualize_and_save_cam(img_path[0], cam, pred_label, actual_label_str, 
                                             case_output_dir, filename, show_plots)
                    
                except Exception as e:
                    print(f"❌ Error processing image {i} for {body_part} {case_type}: {e}")
                    continue
    
    print(f"\n🎉 CAM generation complete for all body parts!")
    print(f"Results saved in: {output_dir}")

# Function to generate summary statistics
def generate_summary_stats(output_dir):
    """Generate summary statistics of CAM generation"""
    print(f"\n📊 CAM Generation Summary:")
    print(f"{'='*50}")
    
    body_parts = ['XR_ELBOW', 'XR_FINGER', 'XR_FOREARM', 'XR_HAND', 
                  'XR_HUMERUS', 'XR_SHOULDER', 'XR_WRIST']
    
    total_cams = 0
    for body_part in body_parts:
        body_part_dir = os.path.join(output_dir, body_part)
        if os.path.exists(body_part_dir):
            normal_dir = os.path.join(body_part_dir, 'normal')
            abnormal_dir = os.path.join(body_part_dir, 'abnormal')
            
            normal_count = 0
            abnormal_count = 0
            
            if os.path.exists(normal_dir):
                for subdir in os.listdir(normal_dir):
                    subdir_path = os.path.join(normal_dir, subdir)
                    if os.path.isdir(subdir_path):
                        normal_count += len([f for f in os.listdir(subdir_path) if f.endswith('.png')])
            
            if os.path.exists(abnormal_dir):
                for subdir in os.listdir(abnormal_dir):
                    subdir_path = os.path.join(abnormal_dir, subdir)
                    if os.path.isdir(subdir_path):
                        abnormal_count += len([f for f in os.listdir(subdir_path) if f.endswith('.png')])
            
            part_total = normal_count + abnormal_count
            total_cams += part_total
            
            if part_total > 0:
                print(f"{body_part:12}: Normal={normal_count:2d}, Abnormal={abnormal_count:2d}, Total={part_total:2d}")
    
    print(f"{'='*50}")
    print(f"Total CAMs generated: {total_cams}")

# Main execution
if __name__ == "__main__":
    # Configure paths (adjust to your setup)
    valid_csv = r"C:\Users\ASUS\OneDrive\Desktop\Mura\MURA-v1.1\valid_labeled_studies.csv"
    root_dir = r"C:\Users\ASUS\OneDrive\Desktop\Mura\MURA-v1.1"
    model_dir = r"C:\Users\ASUS\OneDrive\Desktop\Mura"
    output_dir = "cam_visualizations_all_parts"
    
    # Generate CAMs for all body parts with both normal and abnormal cases
    generate_cams_for_all_body_parts(
        image_csv=valid_csv,
        root_dir=root_dir,
        model_dir=model_dir,
        output_dir=output_dir,
        num_normal=3,      # Number of normal cases per body part
        num_abnormal=3,    # Number of abnormal cases per body part
        show_plots=False   # Set to True if you want to see plots during processing
    )
    
    # Generate summary statistics
    generate_summary_stats(output_dir)
    
    print("\n🎉 Complete pipeline finished!")


Using device: cpu

Processing XR_ELBOW


C:\Users\ASUS\AppData\Local\Temp\ipykernel_14588\1223658375.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


✅ Model loaded successfully for XR_ELBOW

📊 Processing normal cases for XR_ELBOW...
📁 Found 3 normal images for XR_ELBOW


Generating normal CAMs:   0%|                                                                    | 0/3 [00:00<?, ?it/s]C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Saving to directory: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_ELBOW\normal\predicted
Saved original to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_ELBOW\normal\predicted\XR_ELBOW_normal_predicted_0_20250723_125328_original.png
Saved heatmap to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_ELBOW\normal\predicted\XR_ELBOW_normal_predicted_0_20250723_125328_heatmap.png
Saved overlay to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_ELBOW\normal\predicted\XR_ELBOW_normal_predicted_0_20250723_125328_cam_overlay.png


...

Saving to directory: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual
Saved original to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual\XR_WRIST_abnormal_actual_0_20250723_125535_original.png
Saved heatmap to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual\XR_WRIST_abnormal_actual_0_20250723_125535_heatmap.png
Saved overlay to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual\XR_WRIST_abnormal_actual_0_20250723_125535_cam_overlay.png
Saving to directory: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\predicted
Saved original to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\predicted\XR_WRIST_abnormal_predicted_1_20250723_125539_original.png
Saved heatmap to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\predicte

Generating abnormal CAMs:  67%|██████████████████████████████████████▋                   | 2/3 [00:12<00:06,  6.36s/it]

Saving to directory: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual
Saved original to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual\XR_WRIST_abnormal_actual_1_20250723_125541_original.png
Saved heatmap to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual\XR_WRIST_abnormal_actual_1_20250723_125541_heatmap.png
Saved overlay to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual\XR_WRIST_abnormal_actual_1_20250723_125541_cam_overlay.png
Saving to directory: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\predicted
Saved original to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\predicted\XR_WRIST_abnormal_predicted_2_20250723_125545_original.png
Saved heatmap to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\predicte

Generating abnormal CAMs: 100%|██████████████████████████████████████████████████████████| 3/3 [00:19<00:00,  6.35s/it]

Saving to directory: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual
Saved original to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual\XR_WRIST_abnormal_actual_2_20250723_125548_original.png
Saved heatmap to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual\XR_WRIST_abnormal_actual_2_20250723_125548_heatmap.png
Saved overlay to: C:\Users\ASUS\OneDrive\Desktop\Mura\cam_visualizations_all_parts\XR_WRIST\abnormal\actual\XR_WRIST_abnormal_actual_2_20250723_125548_cam_overlay.png

🎉 CAM generation complete for all body parts!
Results saved in: cam_visualizations_all_parts

📊 CAM Generation Summary:
XR_ELBOW    : Normal=18, Abnormal=18, Total=36
XR_FINGER   : Normal=18, Abnormal=18, Total=36
XR_FOREARM  : Normal=18, Abnormal=18, Total=36
XR_HAND     : Normal=18, Abnormal=18, Total=36
XR_HUMERUS  : Normal=18, Abnormal=18, Total=36
XR_SHOULDER : Normal=18, Abnormal=18, 